In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math 

from numba import jit
from numba import cuda

In [25]:
arg_format = ("void(float64, float64, int32, int32, float64[:,:], float64[:,:], "
              "float64[:], float64[:],  float64[:,:], float64[:,:], int32,"
              "float64[:], float64[:], float64[:], float64[:], float64[:])")

@cuda.jit(arg_format)
def Line_Search_Once(decay_rate, step_size, coor_num, momen_num,
                     coor, momentum, measurement, initial_guess,
                     history_loss, history_guess, iter_num,
                     updated_guess, patternCos, patternSin,
                     difference, gradient ):
    
    # Define some useful variables
    grad_norm = 0
    loss = 0
    updated_loss = 0
    holder = 0
    
    ##############################################################################################
    ## Calculate the fourier transition
    ##############################################################################################

    ### 1. Calculate the pattern
    c = cuda.grid(1)
    if c < momen_num :
        patternCos[c] = 0
        patternSin[c] = 0
            
        for m in range(coor_num):
            ## Initialize the variables
            holder = 0
            for l in range(3):
                holder += coor[m, l] * momentum[c , l]
                
            patternCos[c] +=  initial_guess[m]* math.cos(holder)
            patternSin[c] +=  initial_guess[m]* math.sin(holder)
    ### 2 syncthreads
    cuda.syncthreads()

    ##############################################################################################

    
    ##############################################################################################
    ## Calculate the loss function and the difference
    ##############################################################################################

    ### 1. Calculate the pattern
    c = cuda.grid(1)
    if c < momen_num:
        difference[c] = patternCos[c]*patternCos[c] + patternSin[c]*patternSin[c] - measurement[c]
        loss += difference[c] 
        
    ### 2 syncthreads
    cuda.syncthreads()

    ##############################################################################################
    
    
    ##############################################################################################
    ## Calculate the gradient and the norm of the gradient
    ##############################################################################################
    
    ### 1. Calculate the pattern
    c = cuda.grid(1)
    if c < coor_num :
        gradient[c] = 0 # Initialize
        
        for m in range(momen_num):
            holder = 0 # Initialize 
            
            for l in range(3):
                holder += coor[c, l] * momentum[m , l]
                
            gradient[c] += 4*difference[m]* (patternCos[m]*math.cos(holder) +
                                           patternSin[m]*math.cos(holder))
        grad_norm += gradient[c]**2
        
    ### 2 syncthreads
    cuda.syncthreads()

    ##############################################################################################

    
    ##############################################################################################
    ## Beging searching the line
    ##############################################################################################

    ## Notice that when the condition is satified, the function simply return. There
    ## is no need to use a flag to dicide whether to stop.
    
    while True:
        
        ##############################################################################################
        ## update the guess
        ##############################################################################################
        c = cuda.grid(1)    
        if c < coor_num :
            updated_guess[c] = initial_guess[c] - step_size*gradient[c]
            
            # The result has to be positive, other wise reduce the stepsize
            ## ( !!!Attention!!! ) This will introduce singularity into the algorithm and should be fixed later
            if updated_guess[c]<0 :
                step_size = step_size* decay_rate
                continue
        cuda.syncthreads()
        
        ##############################################################################################
        ## Calculate the fourier transition
        ##############################################################################################
        ### 1. Calculate the pattern
        c = cuda.grid(1)
        if c < momen_num :
            patternCos[c] = 0
            patternSin[c] = 0

            for m in range(coor_num):
                ## Initialize the variables
                holder = 0
                for l in range(3):
                    holder += coor[m, l] * momentum[c , l]

                patternCos[c] +=  updated_guess[m]* math.cos(holder)
                patternSin[c] +=  updated_guess[m]* math.sin(holder)
        ### 2 syncthreads
        cuda.syncthreads()

        ##############################################################################################


        ##############################################################################################
        ## Calculate the loss function and the difference
        ##############################################################################################

        ### 1. initialize the variables
        loss = 0

        ### 2. Calculate the pattern
        c = cuda.grid(1)
        if c < momen_num:
            difference[c] = patternCos[c]*patternCos[c] + patternSin[c]*patternSin[c] - measurement[c]
            loss += difference[c] 

        ### 3 syncthreads
        cuda.syncthreads()

        ##############################################################################################
    
        ## Check the condition
        ## If the condition is satified, then return. Otherwise update the step size
        if updated_loss <= loss - step_size/2 *grad_norm :
            
            # update history and guess
            history_loss[iter_num] = updated_loss

            if c < coor_num :
                initial_guess[c] = updated_guess[c]
                history_guess[iter_num, c] = initial_guess[c]
            
            ## No need to
            return
        
        step_size = step_size* decay_rate